In [ ]:
# для colab
import os, sys, subprocess

REPO = "andersonTheCat/witch_practicum"
BRANCH = "main"
DEST = "/content/_repo"

if not os.path.exists(DEST):
    subprocess.run(
        ["git", "clone", "--depth", "1", "-b", BRANCH, f"https://github.com/{REPO}.git", DEST],
        check=True
    )
else:
    subprocess.run(["git", "-C", DEST, "pull", "--ff-only"], check=True)

if DEST not in sys.path:
    sys.path.insert(0, DEST)

print("(=^..^=) repo ready at", DEST)

In [4]:
import numpy as np
from IPython.display import display, Markdown

from meowmeow import (
    Mat, Vec, display_latex, _err,
    _as_array_M, _as_array_v,
    _ops_zero, _ops_inc, _ops_merge,
)

from meowmeowmeow import (
    _vnorm, _mnorm,
    make_x0,
    jacobi_prepare, jacobi_solve,
    _post_metrics, compare_with_gauss,
    _print_system_stats,
    make_dd_system, make_borderline_system, make_bad_system,
)

In [5]:
def _split_B(B: np.ndarray):
    B1 = np.tril(B, k=-1)
    B2 = np.triu(B, k=+1)
    return B1, B2

In [6]:
def seidel_prepare(A_in, b_in):
    B, c, prep = jacobi_prepare(A_in, b_in)
    Barr = _as_array_M(B)
    B1arr, B2arr = _split_B(Barr)
    return Mat(Barr), Mat(B1arr), Mat(B2arr), c, prep

In [7]:
def _seidel_step_with_ops(B: np.ndarray, x_old: np.ndarray, c: np.ndarray, ops):
    n = B.shape[0]
    x_new = x_old.copy()
    for i in range(n):
        s = 0.0
        for j in range(n):
            if j == i:
                continue
            v = x_new[j] if j < i else x_old[j]
            s = s + B[i, j] * v
            _ops_inc(ops, "mul"); _ops_inc(ops, "add")
        x_new[i] = s + c[i]
        _ops_inc(ops, "add")
    return x_new

In [8]:
def _stop_check_seidel(stop_rule: str,
                       A: np.ndarray, b: np.ndarray,
                       Bnorm: float, B2norm: float,
                       x_new: np.ndarray, x_old: np.ndarray,
                       eps: float, vec_norm_kind: str):
    delta = _vnorm(x_new - x_old, vec_norm_kind)

    if stop_rule == "delta":
        return (delta < eps), {"delta": delta}

    if stop_rule == "delta_scaled":
        if (not np.isfinite(Bnorm)) or (not np.isfinite(B2norm)) or (Bnorm >= 1) or (B2norm <= 0):
            return False, {"delta": delta, "eps2": float("nan")}
        eps2 = (1.0 - Bnorm) / B2norm * eps
        return (delta < eps2), {"delta": delta, "eps2": eps2}

    if stop_rule == "aposteriori":
        if (not np.isfinite(Bnorm)) or (not np.isfinite(B2norm)) or (Bnorm >= 1):
            return False, {"delta": delta, "bound": float("inf")}
        bound = (B2norm / (1.0 - Bnorm)) * delta
        return (bound < eps), {"delta": delta, "bound": bound}

    if stop_rule == "residual":
        r = A @ x_new - b
        rn = _vnorm(r, vec_norm_kind)
        return (rn < eps), {"delta": delta, "res": rn}

    raise _err("stop_rule должен быть одним из этих вот: delta / delta_scaled / aposteriori / residual")

In [9]:
def seidel_solve(A_in, b_in, x0: Vec,
                eps: float = 1e-6,
                max_iter: int = 500,
                stop_rule: str = "aposteriori",
                Bnorm_kind: str = "inf",
                vec_norm_kind: str = "2",
                log_each: bool = True):
    A = _as_array_M(A_in)
    b = _as_array_v(b_in)
    n = A.shape[0]

    B, B1, B2, c, prep = seidel_prepare(Mat(A), Vec(b))
    Barr  = _as_array_M(B)
    B2arr = _as_array_M(B2)
    carr  = _as_array_v(c)

    try:
        Bnorm = _mnorm(Barr, Bnorm_kind)
    except Exception:
        Bnorm = float("nan")
    try:
        B2norm = _mnorm(B2arr, Bnorm_kind)
    except Exception:
        B2norm = float("nan")

    ops_iter = _ops_zero()
    ops_total = _ops_merge(prep["ops_prepare"], ops_iter)

    x_old = _as_array_v(x0).copy()
    if x_old.shape != (n,):
        raise _err("x0 должен иметь размерность (n,)")

    history = []
    if log_each:
        display(Markdown(
            f"Зейдель: стоп=`{stop_rule}`, "
            f"$\\|B\\|_{{{Bnorm_kind}}}$=`{Bnorm:.6g}`, "
            f"$\\|B_2\\|_{{{Bnorm_kind}}}$=`{B2norm:.6g}`, "
            f"норма вектора=`{vec_norm_kind}`, eps=`{eps}`"
        ))
        display_latex(Vec(x_old), label=r"x^{(0)}")

    prev_delta = None
    for k in range(0, max_iter):
        x_new = _seidel_step_with_ops(Barr, x_old, carr, ops_iter)
        ok, extra = _stop_check_seidel(stop_rule, A, b, Bnorm, B2norm, x_new, x_old, eps, vec_norm_kind)
        delta = extra.get("delta", float("nan"))

        q_est = None
        if prev_delta is not None and prev_delta > 0:
            q_est = delta / prev_delta

        row = {
            "k": k+1,
            "x": x_new.copy(),
            "delta": float(delta),
            "q_est": (None if q_est is None else float(q_est)),
            **extra
        }
        history.append(row)

        if log_each:
            msg = (
                f"**шаг {k+1}:**  "
                f"$\\|x^{{({k+1})}}-x^{{({k})}}\\|$ = `{delta:.6g}`"
            )
            if q_est is not None:
                msg += f",  q_est ≈ `{q_est:.6g}`"
            if "bound" in extra:
                msg += f",  апост.оценка = `{extra['bound']:.6g}`"
            if "eps2" in extra:
                msg += f",  eps2 = `{extra['eps2']:.6g}`"
            if "res" in extra:
                msg += f",  ||Ax-b|| = `{extra['res']:.6g}`"
            display(Markdown(msg))
            display_latex(Vec(x_new), label=rf"x^{{({k+1})}}")

        if ok:
            x_hat = Vec(x_new)
            ops_total = _ops_merge(prep["ops_prepare"], ops_iter)
            return x_hat, {
                "B": B, "B1": B1, "B2": B2, "c": c,
                "Bnorm": Bnorm, "B2norm": B2norm,
                "ops_prepare": prep["ops_prepare"],
                "ops_iter": ops_iter,
                "ops_total": ops_total,
                "iters": k+1,
                "history": history,
                "stop_rule": stop_rule,
                "Bnorm_kind": Bnorm_kind,
                "vec_norm_kind": vec_norm_kind
            }

        prev_delta = delta
        x_old = x_new

    x_hat = Vec(x_old)
    ops_total = _ops_merge(prep["ops_prepare"], ops_iter)
    return x_hat, {
        "B": B, "B1": B1, "B2": B2, "c": c,
        "Bnorm": Bnorm, "B2norm": B2norm,
        "ops_prepare": prep["ops_prepare"],
        "ops_iter": ops_iter,
        "ops_total": ops_total,
        "iters": max_iter,
        "history": history,
        "stop_rule": stop_rule,
        "Bnorm_kind": Bnorm_kind,
        "vec_norm_kind": vec_norm_kind,
        "warn": "достигнут max_iter, но критерий не сработал"
    }

In [10]:
def solve_for_epsilons_seidel(A: Mat, b: Vec, x0: Vec,
                              eps_list=(1e-3, 1e-6),
                              stop_rule="aposteriori",
                              Bnorm_kind="inf", vec_norm_kind="2",
                              max_iter=500, log_each=True,
                              x_true: Vec | None = None,
                              want_gauss=True,
                              want_compare_jacobi=True):
    B, c, _ = jacobi_prepare(A, b)
    _print_system_stats(A, b, B=B)

    display(Markdown("**запуск итераций (Зейдель)**"))
    for eps in eps_list:
        display(Markdown(f"\nточность eps = `{eps}`"))

        x_hat, info = seidel_solve(A, b, x0, eps=eps, max_iter=max_iter,
                                   stop_rule=stop_rule, Bnorm_kind=Bnorm_kind,
                                   vec_norm_kind=vec_norm_kind, log_each=log_each)

        display(Markdown(f"**итог (Зейдель):** итераций = **{info['iters']}**"))
        display_latex(x_hat, label=r"\hat x")
        _post_metrics(A, b, x_hat, x_true=x_true, vec_norm_kind=vec_norm_kind)

        display(Markdown("**операции (Зейдель):**"))
        display(Markdown(f"подготовка (B,c): `Q = {info['ops_prepare']['total']}`"))
        display(Markdown(f"итерации:         `Q = {info['ops_iter']['total']}`"))
        display(Markdown(f"всего:            `Q = {info['ops_total']['total']}`"))

        if want_gauss:
            compare_with_gauss(A, b, x_hat, info["ops_total"], pivot="col")

        if want_compare_jacobi:
            xj, jinfo = jacobi_solve(A, b, x0, eps=eps, max_iter=max_iter,
                                     stop_rule="aposteriori", Bnorm_kind=Bnorm_kind,
                                     vec_norm_kind=vec_norm_kind, log_each=False)
            display(Markdown(
                f"сравнение с простой итерацией (Якоби): "
                f"итераций = **{jinfo['iters']}**,  всего операций = **{jinfo['ops_total']['total']}**"
            ))

    print("мяу, готово (=^..^=)")

In [11]:
def demo_three_inputs_seidel(n=5):
    display(Markdown("**1) входные данные: добренькая диагонально преобладающая**"))
    A1, b1, x1 = make_dd_system(n, seed=7, strength=2.5)
    B1, c1, _ = jacobi_prepare(A1, b1)
    x0_1 = make_x0("zeros", n, b1, c1)
    solve_for_epsilons_seidel(A1, b1, x0_1, x_true=x1, log_each=True)

    display(Markdown("\n\n**2) входные данные: почти на грани (обычно медленно)**"))
    A2, b2, x2 = make_borderline_system(n, seed=8, slack=1e-2)
    B2, c2, _ = jacobi_prepare(A2, b2)
    x0_2 = make_x0("zeros", n, b2, c2)
    solve_for_epsilons_seidel(A2, b2, x0_2, x_true=x2, log_each=False)

    display(Markdown("\n\n**3) входные данные: плохонькая (может не сходиться)**"))
    A3, b3, x3 = make_bad_system(n, seed=9)
    B3, c3, _ = jacobi_prepare(A3, b3)
    x0_3 = make_x0("zeros", n, b3, c3)
    solve_for_epsilons_seidel(A3, b3, x0_3, x_true=x3, log_each=False, max_iter=50, want_gauss=False)

In [12]:
def main():
    display(Markdown(
        "мяу мяу, что выбираем?\n\n"
        "[1] демка на трёх входных данных (уютно / медленно / неуютно)\n\n"
        "[2] решить свою систему (ручной ввод)\n\n"
        "[3] сгенерировать диагонально преобладающую систему и решить\n"
    ))
    mode = (input("выбор (1/2/3): ").strip() or "1")

    if mode == "1":
        n = int(input("n (по умолчанию 5): ").strip() or "5")
        demo_three_inputs_seidel(n=n)
        return

    if mode == "2":
        from meow import _read_matrix, _read_vector
        display(Markdown("**ввод A:**"))
        A = _read_matrix()
        if not A.is_square():
            raise _err("нужно ввести квадратную A")
        display(Markdown("**ввод b:**"))
        b = _read_vector("b (n чисел): ")

        B, c, _ = jacobi_prepare(A, b)
        n = A.shape[0]

        display(Markdown("теперь нужно выбрать x0: `zeros` / `ones` / `b` / `c` / `rand`"))
        x0_kind = input("x0 kind → ").strip() or "zeros"
        x0 = make_x0(x0_kind, n, b, c)

        display(Markdown(
            "**критерии остановки (Зейдель):**  \n"
            "`delta`: остановка по шагу  $\\|x^{(k)}-x^{(k-1)}\\| < \\varepsilon$  \n"
            "`delta_scaled`: шаг с поправкой  $\\|x^{(k)}-x^{(k-1)}\\| < \\frac{1-\\|B\\|}{\\|B_2\\|}\\,\\varepsilon$ (нужно $\\|B\\|<1$)  \n"
            "`aposteriori`: апост. оценка ошибки  $\\frac{\\|B_2\\|}{1-\\|B\\|}\\,\\|x^{(k)}-x^{(k-1)}\\| < \\varepsilon$ (нужно $\\|B\\|<1$)  \n"
            "`residual`: по невязке  $\\|Ax^{(k)}-b\\| < \\varepsilon$  \n"
        ))
        stop_rule = input("stop (aposteriori/delta_scaled/delta/residual) → ").strip() or "aposteriori"

        eps1 = float(input("eps для 1-го прогона (по умолчанию 1e-3): ").strip() or "1e-3")
        eps2 = float(input("eps для 2-го прогона (по умолчанию 1e-6): ").strip() or "1e-6")

        log_each = (input("печатать каждую итерацию? (y/n, по умолчанию y): ").strip().lower() or "y").startswith("y")
        solve_for_epsilons_seidel(A, b, x0, eps_list=(eps1, eps2),
                                  stop_rule=stop_rule, Bnorm_kind="inf",
                                  vec_norm_kind="2", max_iter=500,
                                  log_each=log_each, x_true=None,
                                  want_gauss=True, want_compare_jacobi=True)
        return

    if mode == "3":
        n = int(input("n (по умолчанию 6): ").strip() or "6")
        seed = int(input("seed (по умолчанию 42): ").strip() or "42")
        strength = float(input("запас диагонального преобладания δ (|a_ii| − ∑_{j≠i}|a_ij|), по умолчанию 2.0: ").strip() or "2.0")
        A, b, x_true = make_dd_system(n, seed=seed, strength=strength)

        B, c, _ = jacobi_prepare(A, b)
        display(Markdown("выбираем x0: `zeros` / `ones` / `b` / `c` / `rand`"))
        x0_kind = input("x0 kind → ").strip() or "zeros"
        x0 = make_x0(x0_kind, n, b, c, seed=seed+1)

        display(Markdown(
            "**критерии остановки (Зейдель):**  \n"
            "`delta`: остановка по шагу  $\\|x^{(k)}-x^{(k-1)}\\| < \\varepsilon$  \n"
            "`delta_scaled`: шаг с поправкой  $\\|x^{(k)}-x^{(k-1)}\\| < \\frac{1-\\|B\\|}{\\|B_2\\|}\\,\\varepsilon$ (нужно $\\|B\\|<1$)  \n"
            "`aposteriori`: апост. оценка ошибки  $\\frac{\\|B_2\\|}{1-\\|B\\|}\\,\\|x^{(k)}-x^{(k-1)}\\| < \\varepsilon$ (нужно $\\|B\\|<1$)  \n"
            "`residual`: по невязке  $\\|Ax^{(k)}-b\\| < \\varepsilon$  \n"
        ))
        stop_rule = input("stop (aposteriori/delta_scaled/delta/residual) [aposteriori]: ").strip() or "aposteriori"
        log_each = (input("печатать каждую итерацию? (y/n, по умолчанию n): ").strip().lower() or "n").startswith("y")

        solve_for_epsilons_seidel(A, b, x0, eps_list=(1e-3, 1e-6),
                                  stop_rule=stop_rule, Bnorm_kind="inf",
                                  vec_norm_kind="2", max_iter=500,
                                  log_each=log_each, x_true=x_true,
                                  want_gauss=True, want_compare_jacobi=True)
        return

    print("мяу мяу, выбери 1/2/3")

In [ ]:
main()